In [1]:
import xarray as xr
import pandas as pd
import numpy as np

import zarr
import numcodecs

import glob
import os
import pathlib
import re

import psutil
import dask
from dask.distributed import Client

In [2]:
def setup_dask_client(
    workload_type="io",        # "cpu", "io", or "mixed"
    max_workers=None,           # Limit max workers (default = all cores)
    reserve_mem_gb=50,          # Memory reserved for system and overhead
    max_mem_gb=None,            # Cap usable memory if needed
    dashboard=True
):
    """
    Setup Dask client with auto-scaling for workload type.

    Parameters:
        workload_type (str): Type of workload - "cpu", "io", or "mixed"
        max_workers (int): Max logical cores to use (default = system max)
        reserve_mem_gb (int): System memory to reserve
        max_mem_gb (int): Cap memory usage (defaults to system memory)
        dashboard (bool): Whether to print the dashboard link

    Returns:
        dask.distributed.Client
    """
    assert workload_type in ("cpu", "io", "mixed"), "Invalid workload_type"

    logical_cores = psutil.cpu_count(logical=True)
    total_memory_gb = psutil.virtual_memory().total / 1e9

    if max_workers is None:
        max_workers = logical_cores

    if max_mem_gb is None:
        max_mem_gb = total_memory_gb

    usable_mem_gb = max_mem_gb - reserve_mem_gb

    # Recommended presets based on workload type
    if workload_type == "cpu":
        threads_per_worker = 1
        n_workers = min(max_workers, logical_cores)
    elif workload_type == "io":
        threads_per_worker = 8
        n_workers = max(1, logical_cores // threads_per_worker)
    else:  # "mixed"
        threads_per_worker = 4
        n_workers = max(1, logical_cores // threads_per_worker)

    memory_per_worker = usable_mem_gb // n_workers

    client = Client(
        n_workers=n_workers,
        threads_per_worker=threads_per_worker,
        memory_limit=f"{int(memory_per_worker)}GB"
    )

    if dashboard:
        print(f"Dask dashboard: {client.dashboard_link}")

    return client

In [3]:
client = setup_dask_client(workload_type="io")
client

Dask dashboard: /node/gadi-cpu-spr-0693.gadi.nci.org.au/18866/proxy/8787/status


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /node/gadi-cpu-spr-0693.gadi.nci.org.au/18866/proxy/8787/status,
Dashboard: /node/gadi-cpu-spr-0693.gadi.nci.org.au/18866/proxy/8787/status,Workers: 26
Total threads: 208,Total memory: 435.86 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33571,Workers: 26
Dashboard: /node/gadi-cpu-spr-0693.gadi.nci.org.au/18866/proxy/8787/status,Total threads: 208
Started: Just now,Total memory: 435.86 GiB
Comm: tcp://127.0.0.1:40875,Total threads: 8
Dashboard: /node/gadi-cpu-spr-0693.gadi.nci.org.au/18866/proxy/35769/status,Memory: 16.76 GiB
Nanny: tcp://127.0.0.1:43031,


# Load all files from single realization

In [4]:
def add_realization_dim(ds):
    # Extract from filename
    match = re.search(r"r\d+i\d+p\d+f\d+", ds.encoding["source"])
    if match:
        realization = match.group()
        return ds.expand_dims(realization=[realization])
    else:
        raise ValueError(f"Could not extract realization from {ds.encoding['source']}")

In [5]:
ds = xr.open_mfdataset(
    "/g/data/hh5/tmp/pa1490/Global_HWs/Data/tas/Model_Data/CMIP6/ssp126/Global_Data/gridded_absolute_temperature/Global_gridded_absolute_temperature_CMIP6_CCCma_CanESM5_ssp126_*.nc",
    concat_dim="realization",
    combine="nested",
    preprocess=add_realization_dim,
    chunks={'lat': -1, 'lon': -1},  # or "auto"
    parallel=True,
    engine="h5netcdf"
)

In [6]:
ds

<xarray.Dataset> Size: 51GB
Dimensions:      (realization: 50, time: 31390, bnds: 2, lon: 128, lat: 64)
Coordinates:
  * realization  (realization) object 400B 'r10i1p1f1' ... 'r9i1p2f1'
  * time         (time) object 251kB 2015-01-01 12:00:00 ... 2100-12-31 12:00:00
  * lon          (lon) float64 1kB 0.0 2.812 5.625 8.438 ... 351.6 354.4 357.2
  * lat          (lat) float64 512B -87.86 -85.1 -82.31 ... 82.31 85.1 87.86
    height       float64 8B 2.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds    (realization, time, bnds) object 25MB dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    lon_bnds     (realization, lon, bnds) float64 102kB dask.array<chunksize=(1, 128, 2), meta=np.ndarray>
    lat_bnds     (realization, lat, bnds) float64 51kB dask.array<chunksize=(1, 64, 2), meta=np.ndarray>
    tas          (realization, time, lat, lon) float32 51GB dask.array<chunksize=(1, 1, 64, 128), meta=np.ndarray>
Attributes: (12/55)
    CDI:                         Climate Data Interface version 2.1.0 (https:...
    source:                      CanESM5 (2019): \naerosol: interactive\natmo...
    institution:                 Canadian Centre for Climate Modelling and An...
    Conventions:                 CF-1.7 CMIP-6.2
    CCCma_model_hash:            24718c8346665b218729640ffe79d263b76456c4
    CCCma_parent_runid:          rc3.1-his10
    ...                          ...
    variable_id:                 tas
    variant_label:               r10i1p1f1
    version:                     v20190429
    license:                     CMIP6 model data produced by The Government ...
    cmor_version:                3.4.0
    CDO:                         Climate Data Operators version 2.1.0 (https:...

# Set the chunking so that each chunk is ~300mb

In [7]:
ds_ch = ds.chunk({'time': 10000, 'realization': 1, 'lat': -1, 'lon': -1})

In [8]:
ds_ch

<xarray.Dataset> Size: 51GB
Dimensions:      (realization: 50, time: 31390, bnds: 2, lon: 128, lat: 64)
Coordinates:
  * realization  (realization) object 400B 'r10i1p1f1' ... 'r9i1p2f1'
  * time         (time) object 251kB 2015-01-01 12:00:00 ... 2100-12-31 12:00:00
  * lon          (lon) float64 1kB 0.0 2.812 5.625 8.438 ... 351.6 354.4 357.2
  * lat          (lat) float64 512B -87.86 -85.1 -82.31 ... 82.31 85.1 87.86
    height       float64 8B 2.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds    (realization, time, bnds) object 25MB dask.array<chunksize=(1, 10000, 2), meta=np.ndarray>
    lon_bnds     (realization, lon, bnds) float64 102kB dask.array<chunksize=(1, 128, 2), meta=np.ndarray>
    lat_bnds     (realization, lat, bnds) float64 51kB dask.array<chunksize=(1, 64, 2), meta=np.ndarray>
    tas          (realization, time, lat, lon) float32 51GB dask.array<chunksize=(1, 10000, 64, 128), meta=np.ndarray>
Attributes: (12/55)
    CDI:                         Climate Data Interface version 2.1.0 (https:...
    source:                      CanESM5 (2019): \naerosol: interactive\natmo...
    institution:                 Canadian Centre for Climate Modelling and An...
    Conventions:                 CF-1.7 CMIP-6.2
    CCCma_model_hash:            24718c8346665b218729640ffe79d263b76456c4
    CCCma_parent_runid:          rc3.1-his10
    ...                          ...
    variable_id:                 tas
    variant_label:               r10i1p1f1
    version:                     v20190429
    license:                     CMIP6 model data produced by The Government ...
    cmor_version:                3.4.0
    CDO:                         Climate Data Operators version 2.1.0 (https:...

# Create a zarr store and save each realization as a 'sub-directory'

In [9]:
output_store = "/g/data/gb02/sg7549/philipp/CCCma_CanESM5.zarr"
os.makedirs(output_store, exist_ok=True)

In [ ]:
for r in ds.realization.values:
    group_path = os.path.join(output_store, r)
    subset = ds_ch.sel(realization=f'{r}')
    
    # === Define compressor ===
    compressor = numcodecs.Blosc(
    cname ='zstd', 
    clevel = 9, 
    shuffle = numcodecs.Blosc.SHUFFLE
    )
        
    # === Define encoding ==
    encoding = {
        var: {"compressor": compressor}
        for var in subset.data_vars
    }
        
    # === Save to Zarr ===
    subset.to_zarr(
        store=group_path,
        consolidated=True,
        encoding=encoding,
        mode="w"
    )
    print(f"Written realization {r} to group: {group_path}")

/g/data/hh5/public/apps/cms_conda/envs/analysis3-24.07/lib/python3.10/site-packages/distributed/client.py:3362: UserWarning: Sending large graph of size 11.41 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


Written realization r10i1p1f1 to group: /g/data/gb02/sg7549/philipp/CCCma_CanESM5.zarr/r10i1p1f1


/g/data/hh5/public/apps/cms_conda/envs/analysis3-24.07/lib/python3.10/site-packages/distributed/client.py:3362: UserWarning: Sending large graph of size 11.41 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


Written realization r10i1p2f1 to group: /g/data/gb02/sg7549/philipp/CCCma_CanESM5.zarr/r10i1p2f1


/g/data/hh5/public/apps/cms_conda/envs/analysis3-24.07/lib/python3.10/site-packages/distributed/client.py:3362: UserWarning: Sending large graph of size 11.41 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


...

Written realization r12i1p1f1 to group: /g/data/gb02/sg7549/philipp/CCCma_CanESM5.zarr/r12i1p1f1


/g/data/hh5/public/apps/cms_conda/envs/analysis3-24.07/lib/python3.10/site-packages/distributed/client.py:3362: UserWarning: Sending large graph of size 11.41 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


Written realization r12i1p2f1 to group: /g/data/gb02/sg7549/philipp/CCCma_CanESM5.zarr/r12i1p2f1


/g/data/hh5/public/apps/cms_conda/envs/analysis3-24.07/lib/python3.10/site-packages/distributed/client.py:3362: UserWarning: Sending large graph of size 11.41 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


Written realization r13i1p1f1 to group: /g/data/gb02/sg7549/philipp/CCCma_CanESM5.zarr/r13i1p1f1


In [7]:
dz = xr.open_zarr("/g/data/gb02/sg7549/philipp/CCCma_CanESM5.zarr/r10i1p1f1")

In [8]:
dz

<xarray.Dataset> Size: 1GB
Dimensions:      (lat: 64, bnds: 2, lon: 128, time: 31390)
Coordinates:
    height       float64 8B ...
  * lat          (lat) float64 512B -87.86 -85.1 -82.31 ... 82.31 85.1 87.86
  * lon          (lon) float64 1kB 0.0 2.812 5.625 8.438 ... 351.6 354.4 357.2
    realization  <U9 36B ...
  * time         (time) object 251kB 2015-01-01 12:00:00 ... 2100-12-31 12:00:00
Dimensions without coordinates: bnds
Data variables:
    lat_bnds     (lat, bnds) float64 1kB dask.array<chunksize=(64, 2), meta=np.ndarray>
    lon_bnds     (lon, bnds) float64 2kB dask.array<chunksize=(128, 2), meta=np.ndarray>
    tas          (time, lat, lon) float32 1GB dask.array<chunksize=(10000, 64, 128), meta=np.ndarray>
    time_bnds    (time, bnds) object 502kB dask.array<chunksize=(10000, 2), meta=np.ndarray>
Attributes: (12/55)
    CCCma_model_hash:            24718c8346665b218729640ffe79d263b76456c4
    CCCma_parent_runid:          rc3.1-his10
    CCCma_pycmor_hash:           33c30511acc319a98240633965a04ca99c26427e
    CCCma_runid:                 rc3.1-s2610
    CDI:                         Climate Data Interface version 2.1.0 (https:...
    CDO:                         Climate Data Operators version 2.1.0 (https:...
    ...                          ...
    table_info:                  Creation Date:(20 February 2019) MD5:374fbe5...
    title:                       CanESM5 output prepared for CMIP6
    tracking_id:                 hdl:21.14100/79fc079c-dd74-4e71-8f4c-133d4dd...
    variable_id:                 tas
    variant_label:               r10i1p1f1
    version:                     v20190429